<a href="https://colab.research.google.com/github/pasancardiffmet/CMP7005-PRAC1/blob/main/ST20319008_CMP7005_PRAC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Beijing Air Quality Analysis


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Set a consistent style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("dark")

In [2]:
# Load the datasets
df_changping = pd.read_csv('PRSA_Data_Changping_20130301-20170228.csv')
df_dongsi = pd.read_csv('PRSA_Data_Dongsi_20130301-20170228.csv')
df_gucheng = pd.read_csv('PRSA_Data_Gucheng_20130301-20170228.csv')
df_huairou = pd.read_csv('PRSA_Data_Huairou_20130301-20170228.csv')

# Add the Site Category column
df_changping['category'] = 'Suburban'
df_dongsi['category'] = 'Urban'
df_gucheng['category'] = 'Industrial'
df_huairou['category'] = 'Rural'

#Merging the Dataset
df_list = [df_changping, df_dongsi, df_gucheng, df_huairou]
merged_df = pd.concat(df_list, ignore_index=True)
merged_df['datetime'] = pd.to_datetime(merged_df[['year', 'month', 'day', 'hour']])
# Rearranging and replacing redundant columns
cols = ['datetime', 'station', 'category', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3',
        'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd','WSPM']
merged_df = merged_df[cols]

merged_df.head()
# Save merged dataset
# merged_df.to_csv('merged_beijing_air_quality.csv', index=False)
# Display the first few rows of each DataFrame
# display(df_changping.head())
# display(df_dongsi.head())
# display(df_gucheng.head())
# display(df_huairou.head())

In [3]:
#Descriptive Statistics
print(f"Total rows in merged dataset: {merged_df.shape[0]}")
print(f"Total columns in merged dataset: {merged_df.shape[1]}")
print("Information about the merged dataset:")
print(merged_df.info(),'\n')
print("Descriptive statistics of the merged dataset:")
print(merged_df.describe(),'\n')
print("\nUnique value counts in categorical columns:")
print("\nUnique values in Station column:")
print(merged_df['station'].value_counts())
print("\nUnique values in Category column:")
print(merged_df['category'].value_counts())
print("\nUnique values in Wind Direction column:")
print(merged_df['wd'].value_counts())
print("\nMissing values and Percentaged per Column:")
missing_values = merged_df.isnull().sum()
missing_percentage = (merged_df.isnull().sum() / len(merged_df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_values, 'Missing Percentage (%)': missing_percentage})
print(missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False))

Total rows in merged dataset: 140256
Total columns in merged dataset: 15
Information about the merged dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140256 entries, 0 to 140255
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  140256 non-null  datetime64[ns]
 1   station   140256 non-null  object        
 2   category  140256 non-null  object        
 3   PM2.5     137133 non-null  float64       
 4   PM10      137963 non-null  float64       
 5   SO2       137478 non-null  float64       
 6   NO2       135681 non-null  float64       
 7   CO        132715 non-null  float64       
 8   O3        137108 non-null  float64       
 9   TEMP      140081 non-null  float64       
 10  PRES      140083 non-null  float64       
 11  DEWP      140079 non-null  float64       
 12  RAIN      140087 non-null  float64       
 13  wd        139577 non-null  object        
 14  WSPM      140108 non-

In [4]:
#Handling the numerical missing values in the numerical columns

#Since these are time series data, and the number of pollutends are likely be similar to the previous value in the time series (assuming no drastic changes)
#filling with the previous valuee with ffill makes the most sense
numerical_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
for col in numerical_cols:
    merged_df[col] = merged_df[col].ffill()

# Fill missing values in 'wd' categorical column with the mode (most common item)
merged_df['wd'] = merged_df['wd'].fillna(merged_df['wd'].mode()[0])

#Drop any duplicate columns if there are any
merged_df.drop_duplicates()
merged_df
# Save the processed merged dataset
merged_df.to_csv('merged_beijing_air_quality.csv', index=False)

#Data Analysis

,0
datetime,0
station,0
category,0
PM2.5,0
PM10,0
SO2,0
NO2,0
CO,0
O3,0
TEMP,0
